In [ ]:
!pip install tkseem

## Question Answering

### Download and Prepare Data

In [ ]:
!wget https://dl.fbaipublicfiles.com/MLQA/MLQA_V1.zip
!unzip MLQA_V1.zip

### Prepare Data

In [1]:
import json 

def read_data(file_path, max_context_size = 100):
  # Read dataset
  with open(file_path) as f:
      data = json.load(f)

  contexts = []
  questions = []
  answers = []
  labels = []
  
  for i in range(len(data['data'])):

    paragraph_object = data['data'][i]["paragraphs"]
    
    for j in range(len(paragraph_object)):

      context_object = paragraph_object[j]
      context_text = context_object['context']

      if len(context_text.split()) > max_context_size:
        continue
      for k in range(len(context_object['qas'])):

        question_object = context_object['qas'][k]
        question_text = question_object['question']
        
        answer_object = question_object['answers'][0]
        answer_text = answer_object['text']
        answer_start = answer_object['answer_start']
        answer_end = answer_start + len(answer_text)

        answer_start = len(context_text[:answer_start].split())
        answer_end = answer_start + len(answer_text.split())
        if answer_end >= max_context_size:
          answer_end = max_context_size -1
        labels.append([answer_start, answer_end])
        questions.append(question_text)
        contexts.append(context_text)
        answers.append(answer_text)
  
  with open('train_contexts.txt', 'w') as f:
    f.write(('\n').join(contexts))
  with open('train_questions.txt', 'w') as f:
    f.write(('\n').join(questions))
  return {'qas':questions, 'ctx':contexts, 'ans':answers, 'lbl':labels}

In [2]:
train_data = read_data('MLQA_V1/test/test-context-ar-question-ar.json')

In [3]:
for i in range(10):
  print(train_data['qas'][i])
  print(train_data['ctx'][i])
  print(train_data['ans'][i])
  print("==============")

ما الذي جعل شريط الاختبار للطائرة؟
بحيرة جرووم كانت تستخدم للقصف المدفعي والتدريب علي المدفعية خلال الحرب العالمية الثانية، ولكن تم التخلي عنها بعد ذلك حتى نيسان / أبريل 1955، عندما تم اختياره من قبل فريق لوكهيد اسكنك كموقع مثالي لاختبار لوكهيد يو-2 - 2 طائرة التجسس. قاع البحيرة قدم الشريط المثالية التي يمكن عمل اختبارات الطائرات المزعجة، ارتفاع سلسلة جبال وادي الإيمجرانت ومحيط NTS يحمي موقع الاختبار من أعين المتطفلين والتدخل الخارجي.
قاع البحيرة
من كان يرافق طائرة يو -2 عند التسليم؟
شيدت لوكهيد قاعدة مؤقتة في الموقع، ثم عرفت باسم الموقع الثاني أو "المزرعة"، التي تتألف من أكثر بقليل من بضعة مخابئ، وحلقات عمل ومنازل متنقلة لفريقها الصغير. في ثلاثة أشهر فقط شيد مدرج طوله 5000  ودخل الخدمة بحلول تموز / يوليو 1955. حصلت المزرعة على تسليم أول يو 2 في 24 يوليو، 1955 من بوربانك على سي 124 جلوب ماستر الثاني طائرة شحن، يرافقه فنيي وكهيد على دي سي 3. انطلق أول يو - 2 من الجرووم في 4 أغسطس، 1955. بدأت عمليات تحليق أسطول يو 2 تحت سيطرة وكالة المخابرات المركزية الأمريكية في الأجواء السوفياتية بحلول

### Imports

In [4]:
import re
import nltk
import time
import numpy as np
import tkseem as tk
import tensorflow as tf
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

### Tokenization

In [5]:
qa_tokenizer = tk.WordTokenizer()
qa_tokenizer.train('train_questions.txt')
print('Vocab size ', qa_tokenizer.vocab_size)

cx_tokenizer = tk.WordTokenizer()
cx_tokenizer.train('train_contexts.txt')
print('Vocab size ', cx_tokenizer.vocab_size)

train_inp_data = qa_tokenizer.encode_sentences(train_data['qas'])
train_tar_data = cx_tokenizer.encode_sentences(train_data['ctx'])
train_tar_lbls = train_data['lbl']
train_inp_data.shape, train_tar_data.shape

Training WordTokenizer ...
Vocab size  8883
Training WordTokenizer ...
Vocab size  10000


((2936, 23), (2936, 100))

### Create Dataset

In [6]:
BATCH_SIZE = 64
BUFFER_SIZE = len(train_inp_data)

dataset = tf.data.Dataset.from_tensor_slices((train_inp_data, train_tar_data, train_tar_lbls)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

### Create Encoder and Decoder

In [7]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output = self.gru(x, initial_state = hidden)
        return output

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))
  
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, output_sz):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding_dim = embedding_dim
        self.output_sz = output_sz
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=False,
                                       recurrent_initializer='glorot_uniform')
        self.fc11 = tf.keras.layers.Dense(embedding_dim)
        self.fc12 = tf.keras.layers.Dense(output_sz)

        self.fc21 = tf.keras.layers.Dense(embedding_dim)
        self.fc22 = tf.keras.layers.Dense(output_sz)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        x = self.gru(x, initial_state = hidden)
        x1 = self.fc11(x)
        x2 = self.fc21(x)

        x1 = self.fc12(x1)
        x2 = self.fc22(x2)
        return [x1, x2]

def loss_fn(true, pred):
  cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  return (cross_entropy(true[:,0:1], pred[0]) + cross_entropy(true[:,1:2], pred[1]))/2

### Training

In [41]:
units = 1024
embedding_dim = 256
max_length_inp = train_inp_data.shape[1]
max_length_tar = train_tar_data.shape[1]
vocab_tar_size = cx_tokenizer.vocab_size
vocab_inp_size = qa_tokenizer.vocab_size
steps_per_epoch = len(train_inp_data) // BATCH_SIZE
decoder = Decoder(vocab_tar_size, embedding_dim, units, max_length_tar)
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

optim = tf.optimizers.Adam()

In [42]:
epochs = 25
for epoch in range(epochs):
  enc_hidden = encoder.initialize_hidden_state()
  epoch_loss = 0
  
  for idx, (inp, tar, true) in enumerate(dataset):
    with tf.GradientTape() as tape:
        hidden = encoder(inp, enc_hidden)
        pred = decoder(tar, hidden)
        loss  = loss_fn(true, pred)
    variables = decoder.trainable_variables + encoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optim.apply_gradients(zip(gradients, variables))
    epoch_loss += loss.numpy()
  print(f"Epoch {epoch} loss: {epoch_loss/steps_per_epoch:.3f}")

Epoch 0 loss: 4.386
Epoch 1 loss: 4.264
Epoch 2 loss: 4.238
Epoch 3 loss: 4.105
Epoch 4 loss: 3.932
Epoch 5 loss: 3.758
Epoch 6 loss: 3.643
Epoch 7 loss: 3.548
Epoch 8 loss: 3.456
Epoch 9 loss: 3.382
Epoch 10 loss: 3.285
Epoch 11 loss: 3.215
Epoch 12 loss: 3.141
Epoch 13 loss: 3.047
Epoch 14 loss: 2.916
Epoch 15 loss: 2.831
Epoch 16 loss: 2.748
Epoch 17 loss: 2.614
Epoch 18 loss: 2.462
Epoch 19 loss: 2.306
Epoch 20 loss: 2.126
Epoch 21 loss: 1.944
Epoch 22 loss: 1.770
Epoch 23 loss: 1.637
Epoch 24 loss: 1.414


### evaluation

In [43]:
def answer(question_txt, context_txt, answer_txt_tru):
    question = qa_tokenizer.encode_sentences([question_txt], out_length = max_length_inp)
    context = cx_tokenizer.encode_sentences([context_txt], out_length = max_length_tar)
    question = tf.convert_to_tensor(question)
    context = tf.convert_to_tensor(context)
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_hidden = encoder(question, hidden)
    pred = decoder(context, enc_hidden)

    start = tf.argmax(pred[0], axis = -1).numpy()[0]
    end = tf.argmax(pred[1], axis = -1).numpy()[0]
    
    if start >= len(context_txt.split()):
      start = len(context_txt.split()) - 1
    if end >= len(context_txt.split()):
      end = len(context_txt.split()) - 1
    
    # if one word prediction
    if end == start:
      end += 1
    answer_txt = (' ').join(context_txt.split()[start:end])
    
    print("Question : ", question_txt)
    print("Context  : ",context_txt)
    print("Pred Answer : ",answer_txt)
    print("True Answer : ", answer_txt_tru)
    print("======================")

In [44]:
answer("في أي عام توفي وليام ؟", "توفي وليام في عام 1990", "1990")
answer("ماهي عاصمة البحرين ؟", "عاصمة البحرين هي المنامة", "المنامة")
answer("في أي دولة ولد جون ؟", "ولد في فرنسا عام 1988", "فرنسا")
answer("أين تركت الهاتف ؟", "تركت الهاتف فوق الطاولة", "فوق الطاولة")

Question :  في أي عام توفي وليام ؟
Context  :  توفي وليام في عام 1990
Pred Answer :  1990
True Answer :  1990
Question :  ماهي عاصمة البحرين ؟
Context  :  عاصمة البحرين هي المنامة
Pred Answer :  المنامة
True Answer :  المنامة
Question :  في أي دولة ولد جون ؟
Context  :  ولد في فرنسا عام 1988
Pred Answer :  1988
True Answer :  فرنسا
Question :  أين تركت الهاتف ؟
Context  :  تركت الهاتف فوق الطاولة
Pred Answer :  الطاولة
True Answer :  فوق الطاولة
